# Challenge 1 - T-test

In statistics, t-test is used to test if two data samples have a significant difference between their means. There are two types of t-test:

* **Student's t-test** (a.k.a. independent or uncorrelated t-test). This type of t-test is to compare the samples of **two independent populations** (e.g. test scores of students in two different classes). `scipy` provides the [`ttest_ind`](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.ttest_ind.html) method to conduct student's t-test.

* **Paired t-test** (a.k.a. dependent or correlated t-test). This type of t-test is to compare the samples of **the same population** (e.g. scores of different tests of students in the same class). `scipy` provides the [`ttest_re`](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.ttest_rel.html) method to conduct paired t-test.

Both types of t-tests return a number which is called the **p-value**. If p-value is below 0.05, we can confidently declare the null-hypothesis is rejected and the difference is significant. If p-value is between 0.05 and 0.1, we may also declare the null-hypothesis is rejected but we are not highly confident. If p-value is above 0.1 we do not reject the null-hypothesis.

Read more about the t-test in [this article](http://b.link/test50) and [this Quora](http://b.link/unpaired97). Make sure you understand when to use which type of t-test. 

In [146]:
# Import libraries
import pandas as pd
import scipy
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats
from scipy import interpolate
from scipy.interpolate import interp1d

# Librarie for hypothesis testing:
from scipy.stats import ttest_1samp, ttest_rel, ttest_ind
from scipy.stats import f_oneway

from scipy.stats import uniform
from scipy.stats import expon
from scipy.stats import norm
from scipy.stats import poisson
from scipy.stats import binom
from scipy.stats import ttest_ind

#### Import dataset

In this challenge we will work on the Pokemon dataset. The goal is to test whether different groups of pokemon (e.g. Legendary vs Normal, Generation 1 vs 2, single-type vs dual-type) have different stats (e.g. HP, Attack, Defense, etc.).

In [147]:
# Your code here:
pokemon = pd.read_csv('Pokemon.csv')
pokemon.head(20)

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
5,5,Charmeleon,Fire,NaN,405,58,64,58,80,65,80,1,False
6,6,Charizard,Fire,Flying,534,78,84,78,109,85,100,1,False
7,6,CharizardMega Charizard X,Fire,Dragon,634,78,130,111,130,85,100,1,False
8,6,CharizardMega Charizard Y,Fire,Flying,634,78,104,78,159,115,100,1,False
9,7,Squirtle,Water,NaN,314,44,48,65,50,64,43,1,False


In [18]:
pokemon.describe()

,#,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation
count,800.000000,800.00000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.00000
mean,362.813750,435.10250,69.258750,79.001250,73.842500,72.820000,71.902500,68.277500,3.32375
std,208.343798,119.96304,25.534669,32.457366,31.183501,32.722294,27.828916,29.060474,1.66129
min,1.000000,180.00000,1.000000,5.000000,5.000000,10.000000,20.000000,5.000000,1.00000
25%,184.750000,330.00000,50.000000,55.000000,50.000000,49.750000,50.000000,45.000000,2.00000
50%,364.500000,450.00000,65.000000,75.000000,70.000000,65.000000,70.000000,65.000000,3.00000
75%,539.250000,515.00000,80.000000,100.000000,90.000000,95.000000,90.000000,90.000000,5.00000
max,721.000000,780.00000,255.000000,190.000000,230.000000,194.000000,230.000000,180.000000,6.00000


#### First we want to define a function with which we can test the means of a feature set of two samples. 

In the next cell you'll see the annotations of the Python function that explains what this function does and its arguments and returned value. This type of annotation is called **docstring** which is a convention used among Python developers. The docstring convention allows developers to write consistent tech documentations for their codes so that others can read. It also allows some websites to automatically parse the docstrings and display user-friendly documentations.

Follow the specifications of the docstring and complete the function.

In [19]:
# numpy.random.normal(loc=0.0, scale=1.0, size=None)¶
# loc = mean
# scale = std

In [107]:
def t_test_features(s1, s2, features=['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed', 'Total']):
    """Test means of a feature set of two samples
    
    Args:
        s1 (dataframe): sample 1
        s2 (dataframe): sample 2
        features (list): an array of features to test
    
    Returns:
        dict: a dictionary of t-test scores for each feature where the feature name is the key and the p-value is the value
    """
    # Your code here
    
    t_test_ind = ttest_ind(s1[features], s2[features], equal_var=True)
    results = {key:value[1] for key, value in zip(features, t_test_ind)}
    return results

# This is with my own function. However, since the t_test_ind is returning an array, this doesn´t work.
# I haven´t been able to solve it

In [112]:
# With the function given:

def t_test_features_2(s1, s2, features=['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed', 'Total']):
    results = {i: ttest_ind(s1[i],s2[i])[1] for i in features}
    return results

#### Using the `t_test_features` function, conduct t-test for Lengendary vs non-Legendary pokemons.

*Hint: your output should look like below:*

```
{'HP': 1.0026911708035284e-13,
 'Attack': 2.520372449236646e-16,
 'Defense': 4.8269984949193316e-11,
 'Sp. Atk': 1.5514614112239812e-21,
 'Sp. Def': 2.2949327864052826e-15,
 'Speed': 1.049016311882451e-18,
 'Total': 9.357954335957446e-47}
 ```

In [38]:
# Your code here
# 
filter = (pokemon['Legendary'] == True ) 
legendary_sample = pokemon[filter]
legendary_sample

filter = (pokemon['Legendary'] == False ) 
legendary_sample_2 = pokemon[filter]
legendary_sample_2

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
787,711,GourgeistSuper Size,Ghost,Grass,494,85,100,122,58,75,54,6,False
788,712,Bergmite,Ice,NaN,304,55,69,85,32,35,28,6,False
789,713,Avalugg,Ice,NaN,514,95,117,184,44,46,28,6,False
790,714,Noibat,Flying,Dragon,245,40,30,35,45,40,55,6,False


In [113]:
t_test_features_2(legendary_sample, legendary_sample_2, features=['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed', 'Total'])

{'HP': 3.330647684846191e-15,
 'Attack': 7.827253003205333e-24,
 'Defense': 1.5842226094427255e-12,
 'Sp. Atk': 6.314915770427266e-41,
 'Sp. Def': 1.8439809580409333e-26,
 'Speed': 2.3540754436897763e-21,
 'Total': 3.0952457469652825e-52}

#### From the test results above, what conclusion can you make? Do Legendary and non-Legendary pokemons have significantly different stats on each feature?

In [ ]:
# Your comment here
# All the p values are very low, below 0.01, which means that the H0 hypothesis that the distributions are the same can be rejected.
# We can therefore assume that the distributions are not the same.

#### Next, conduct t-test for Generation 1 and Generation 2 pokemons.

In [61]:
# Your code here
filter = (pokemon['Generation'] == 1 ) 
generation_sample = pokemon[filter]
generation_sample

filter = (pokemon['Generation'] == 2 ) 
generation_sample_2 = pokemon[filter]
generation_sample_2


,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
166,152,Chikorita,Grass,NaN,318,45,49,65,49,65,45,2,False
167,153,Bayleef,Grass,NaN,405,60,62,80,63,80,60,2,False
168,154,Meganium,Grass,NaN,525,80,82,100,83,100,80,2,False
169,155,Cyndaquil,Fire,NaN,309,39,52,43,60,50,65,2,False
170,156,Quilava,Fire,NaN,405,58,64,58,80,65,80,2,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,248,Tyranitar,Rock,Dark,600,100,134,110,95,100,61,2,False
268,248,TyranitarMega Tyranitar,Rock,Dark,700,100,164,150,95,120,71,2,False
269,249,Lugia,Psychic,Flying,680,106,90,130,90,154,110,2,True
270,250,Ho-oh,Fire,Flying,680,106,130,90,110,154,90,2,True


In [114]:
t_test_features_2(generation_sample, generation_sample_2, features)

{'HP': 0.13791881412813622,
 'Attack': 0.24050968418101457,
 'Defense': 0.5407630349194362,
 'Sp. Atk': 0.141197881763315,
 'Sp. Def': 0.16781226231606386,
 'Speed': 0.0028356954812578704,
 'Total': 0.559914064901444}

#### What conclusions can you make?

In [ ]:
# Your comment here
# The P values for the Generation and Generation 2 are above 0.5. This means that the H0 hypothesis that the distributions are the
# the same cannot be rejected. Therefore these distributions are the same.

#### Compare pokemons who have single type vs those having two types.

In [136]:
# Your code here
filter = (pokemon['Type 1'].notnull()) 
type_sample = pokemon[filter]
type_sample

filter = (pokemon['Type 2'].notnull()) 
type_sample_2 = pokemon[filter]
type_sample_2


,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
6,6,Charizard,Fire,Flying,534,78,84,78,109,85,100,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True


In [137]:
t_test_features_2(type_sample, type_sample_2, features)

{'HP': 0.3518259206303874,
 'Attack': 0.03630940922862199,
 'Defense': 0.002396942445099732,
 'Sp. Atk': 0.035039978160107364,
 'Sp. Def': 0.030147845116600985,
 'Speed': 0.2009718185618369,
 'Total': 0.0031738961271390136}

In [133]:
# Checking the different types
pokemon["Type 2"].unique()

array(['Poison', nan, 'Flying', 'Dragon', 'Ground', 'Fairy', 'Grass',
       'Fighting', 'Psychic', 'Steel', 'Ice', 'Rock', 'Dark', 'Water',
       'Electric', 'Fire', 'Ghost', 'Bug', 'Normal'], dtype=object)

#### What conclusions can you make?

In [ ]:
# Your comment here

# The P values per column are different. For HP/Speed we can assume that the distribution is the same.
# For the other items, with a 95 % interval, the distribution seems different.

#### Now, we want to compare whether there are significant differences of `Attack` vs `Defense`  and  `Sp. Atk` vs `Sp. Def` of all pokemons. Please write your code below.

*Hint: are you comparing different populations or the same population?*

In [144]:
# Your code here
# I´m not sure how to solve this one. We are comparing different populations, 
# but I don´t know on which item to compare them.

features_3=['Attack']

def t_test_features_3(s1, s2, features_3=['Attack']):
    results = {i: ttest_ind(s1[i],s2[i])[1] for i in features_3}
    return results

attack = pokemon['Attack']
defense = pokemon["Defense"]

t_test_features_3(attack, defense, features_3)

KeyError: 'Attack'

#### What conclusions can you make?

In [ ]:
# Your comment here